<h3>Import Libraries and Data</h3>

In [146]:
import pandas as pd
from sqlalchemy import create_engine

In [163]:
# Store csv data into DataFrame
csv_file = "./data_sources/combined_data_clean.csv"
combined_data_df = pd.read_csv(csv_file)
combined_data_df.head()

,Name,county_state,date1,Serial,date2,Pre_rally_beg,pre_rally_end,post_rally_beg,post_rally_end,average_growth_new_cases,pre_rally_rate,post_rally_rate,post_rally_growth_new_cases,post_rally_change_in_growth,pre_rally_rate.1,post_rally_rate.1,post_rally_growth_deaths
0,Joe Biden,Milwaukee_Wisconsin,20200903,Milwaukee_Wisconsin_20200903,9/3/2020,8/20/2020,9/3/2020,9/3/2020,9/17/2020,8.8%,330503,399423,20.85%,135.99%,6777,7619,12.42%
1,Donald Trump,Westmoreland_Pennsylvania,20200903,Westmoreland_Pennsylvania_20200903,9/3/2020,8/20/2020,9/3/2020,9/3/2020,9/17/2020,28.6%,24721,28982,17.24%,-39.69%,686,736,7.29%
2,Joe Biden,Dauphin_Pennsylvania,20200907,Dauphin_Pennsylvania_20200907,9/7/2020,8/24/2020,9/7/2020,9/7/2020,9/21/2020,-2.5%,46296,53798,16.20%,-746.35%,2284,2543,11.34%
3,Donald Trump,Forsyth_North Carolina,20200908,Forsyth_North Carolina_20200908,9/8/2020,8/25/2020,9/8/2020,9/8/2020,9/22/2020,-3.4%,86603,99921,15.38%,-555.72%,1068,1342,25.66%
4,Donald Trump,Palm Beach_Florida,20200908,Palm Beach_Florida_20200908,9/8/2020,8/25/2020,9/8/2020,9/8/2020,9/22/2020,-11.9%,589620,665859,12.93%,-208.91%,15758,18412,16.84%


<h3>Clean and Transform Data</h3>

In [164]:
# Rename combined_data_df columns to match schema
combined_data_df.rename(columns={'Name': 'name', 'date2': 'date', 'Pre_rally_beg': 'pre_rally_date', 
                                'post_rally_end': 'post_rally_date', 'average_growth_new_cases': 'pre_rally_case_growth_rate',
                                'pre_rally_rate': 'pre_rally_num_cases', 'post_rally_rate': 'post_rally_num_cases',
                                'post_rally_growth_new_cases': 'post_rally_case_growth_rate',
                                'post_rally_change_in_growth': 'case_growth_rate_change',
                                'pre_rally_rate.1': 'pre_rally_num_deaths',
                                'post_rally_rate.1': 'post_rally_num_deaths',
                                'post_rally_growth_deaths': 'death_growth_rate_change'}, inplace=True)
# Convert percent to float
combined_data_df['pre_rally_case_growth_rate'] = combined_data_df['pre_rally_case_growth_rate'].str.rstrip('%').astype('float') / 100.0
combined_data_df['post_rally_case_growth_rate'] = combined_data_df['post_rally_case_growth_rate'].str.rstrip('%').astype('float') / 100.0
combined_data_df['case_growth_rate_change'] = combined_data_df['case_growth_rate_change'].str.rstrip('%').astype('float') / 100.0
combined_data_df['death_growth_rate_change'] = combined_data_df['death_growth_rate_change'].str.rstrip('%').astype('float') / 100.0

# print(combined_data_df)

<h3>Normalize Candidates</h3>

In [165]:
rally_df = pd.DataFrame() # Initialize to lock printed table on re-run
rally_df = combined_data_df[['county_state', 'date', 'case_growth_rate_change', 'death_growth_rate_change']].copy()

rally_candidate_df = combined_data_df[['name']].copy()
print(rally_candidate_df)

candidate_list = []
rally_candidate = []


# Candidates are NOT hard-coded, we allow for any variable number of candidates.

# Loop through candidate_df, if current candidate is not in candidate list, add to list.
# Add and populate new candidate_ID column in rally_df.
for row in rally_candidate_df.itertuples():
    if not row[1] in candidate_list: # If candidate is not already in candidate_list,
        candidate_list.append(row[1]) # Then append candidate to candidate_list.
    rally_candidate.append(candidate_list.index(row[1]) + 1) # Track candidate_id foreign key. +1 because pgAdmin candidate table index starts at 1.

rally_df = rally_df.assign(candidate_id = rally_candidate) # New column. Assign rally candidate, Biden or Trump, to rally.

# print(candidate_list)

candidate_df = pd.DataFrame(candidate_list,columns=['name'])

# print(rally_candidate)
print(candidate_df)

rally_df.head()

             name
0       Joe Biden
1    Donald Trump
2       Joe Biden
3    Donald Trump
4    Donald Trump
..            ...
102  Donald Trump
103  Donald Trump
104  Donald Trump
105     Joe Biden
106     Joe Biden

[107 rows x 1 columns]
           name
0     Joe Biden
1  Donald Trump


,county_state,date,case_growth_rate_change,death_growth_rate_change,candidate_id
0,Milwaukee_Wisconsin,9/3/2020,1.3599,0.1242,1
1,Westmoreland_Pennsylvania,9/3/2020,-0.3969,0.0729,2
2,Dauphin_Pennsylvania,9/7/2020,-7.4635,0.1134,1
3,Forsyth_North Carolina,9/8/2020,-5.5572,0.2566,2
4,Palm Beach_Florida,9/8/2020,-2.0891,0.1684,2


<h3>Connect to Local Database</h3>

In [155]:
rds_connection_string = "postgres:password!@localhost:5432/covid_campaign" # Must have existing 'covid_campaign' database in pgAdmin
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['rally', 'candidate', 'pre_rally_covid_stats', 'post_rally_covid_stats']

<h3>Create Database Tables</h3>

Execute the following SQL code through the query editor in pgAdmin:
``` mysql
DROP TABLE IF EXISTS post_rally_covid_stats CASCADE; -- Drop and import/create in reverse orders to satisfy constraints
DROP TABLE IF EXISTS pre_rally_covid_stats CASCADE;
DROP TABLE IF EXISTS candidate CASCADE;
DROP TABLE IF EXISTS rally CASCADE;

CREATE TABLE rally (
	rally_id SERIAL PRIMARY KEY,
	candidate_id INT,
	county_state TEXT,
	date DATE, -- Date of rally, end of pre window, start of post window
	pre_rally_covid_stats_id INT,
	post_rally_covid_stats_id INT,
	case_growth_rate_change DECIMAL,
	death_growth_rate_change DECIMAL
);

CREATE TABLE candidate (
	candidate_id SERIAL PRIMARY KEY,
	name TEXT
);

CREATE TABLE pre_rally_covid_stats (
	pre_rally_covid_stats_id SERIAL PRIMARY KEY,
	pre_rally_date DATE, -- Pre window start date
	pre_rally_num_cases INT,
	pre_rally_case_growth_rate DECIMAL,
	pre_rally_num_deaths INT
);

CREATE TABLE post_rally_covid_stats (
	post_rally_covid_stats_id SERIAL PRIMARY KEY,
	post_rally_date DATE, -- Post window end date
	post_rally_num_cases INT,
	post_rally_case_growth_rate DECIMAL,
	post_rally_num_deaths INT
);
```

<h3>Build rally_df, pre_rally_covid_stats_df, post_rally_covid_stats_df</h3>

In [166]:
# Add foreign key columns
rally_df['pre_rally_covid_stats_id'] = range(1, len(rally_df) + 1)
rally_df['post_rally_covid_stats_id'] = range(1, len(rally_df) + 1) # pgAdmin table index starts at 1
rally_df.head()

,county_state,date,case_growth_rate_change,death_growth_rate_change,candidate_id,pre_rally_covid_stats_id,post_rally_covid_stats_id
0,Milwaukee_Wisconsin,9/3/2020,1.3599,0.1242,1,1,1
1,Westmoreland_Pennsylvania,9/3/2020,-0.3969,0.0729,2,2,2
2,Dauphin_Pennsylvania,9/7/2020,-7.4635,0.1134,1,3,3
3,Forsyth_North Carolina,9/8/2020,-5.5572,0.2566,2,4,4
4,Palm Beach_Florida,9/8/2020,-2.0891,0.1684,2,5,5


In [167]:
pre_rally_covid_stats_df = combined_data_df[['pre_rally_date', 'pre_rally_num_cases', 
                                             'pre_rally_case_growth_rate', 'pre_rally_num_deaths']].copy()
pre_rally_covid_stats_df.head()

,pre_rally_date,pre_rally_num_cases,pre_rally_case_growth_rate,pre_rally_num_deaths
0,8/20/2020,330503,0.088,6777
1,8/20/2020,24721,0.286,686
2,8/24/2020,46296,-0.025,2284
3,8/25/2020,86603,-0.034,1068
4,8/25/2020,589620,-0.119,15758


In [168]:
post_rally_covid_stats_df = combined_data_df[['post_rally_date', 'post_rally_num_cases', 
                                             'post_rally_case_growth_rate', 'post_rally_num_deaths']].copy()
post_rally_covid_stats_df.head()

,post_rally_date,post_rally_num_cases,post_rally_case_growth_rate,post_rally_num_deaths
0,9/17/2020,399423,0.2085,7619
1,9/17/2020,28982,0.1724,736
2,9/21/2020,53798,0.1620,2543
3,9/22/2020,99921,0.1538,1342
4,9/22/2020,665859,0.1293,18412


<h3>Load DataFrames Into Database</h3>

In [169]:
# Load cleaned pre_stats and post_stats DataFrames into database
pre_rally_covid_stats_df.to_sql(name='pre_rally_covid_stats', con=engine, if_exists='append', index=False)
post_rally_covid_stats_df.to_sql(name='post_rally_covid_stats', con=engine, if_exists='append', index=False)

In [170]:
# Load cleaned rally and candidate DataFrames into database
rally_df.to_sql(name='rally', con=engine, if_exists='append', index=False)
candidate_df.to_sql(name='candidate', con=engine, if_exists='append', index=False)